# GENERAL: Strategic Game AI Training

GPU-accelerated training using AlphaZero-style RL. Optimizations enabled for Colab (T4) and Local (RTX) environments.

This notebook uses the optimized `GenGameAI` codebase with:
- **Non-blocking Inference** (Asyncio + ThreadPool)
- **Efficient Data Loading** (DataLoader with pinned memory)
- **Auto-Configuration** (Detects GPU VRAM and CPU cores)

## 1. Setup

In [ ]:
!git clone https://github.com/Tanish-2006/Generals.git
%cd Generals

In [ ]:
!python3.11 -m pip install torch numpy

In [ ]:
%cd ..

## 2. Configuration & Hardware Detection

In [ ]:
import sys
sys.path.insert(0, '.')
import torch
from config import TRAINING, NETWORK

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("WARNING: No GPU detected.")

print("\nAuto-Detected Configuration:")
print(f"  Workers: {TRAINING.num_workers}")
print(f"  Batch Size: {TRAINING.batch_size}")
print(f"  Games/Iter: {TRAINING.games_per_iter}")

## 3. Upload Previous Model (Optional)
If you have a `model_latest.pth` or `model_old.pth` from a previous run, upload it here to resume training.

In [ ]:
from google.colab import files
from pathlib import Path

CHECKPOINT_DIR = Path("data/checkpoints")
CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)

print("Upload model_latest.pth or model_old.pth if you have one:")
uploaded = files.upload()

for filename in uploaded.keys():
    target_path = CHECKPOINT_DIR / filename
    with open(target_path, 'wb') as f:
        f.write(uploaded[filename])
    print(f"Saved {filename} to {target_path}")

## 4. Training Loop
Runs the main optimized training loop. Supports resuming if models exist.

In [4]:
from main import main_loop

await main_loop(max_iterations=20)

Game 5/20 → Winner: B
Game 6/20 → Winner: A
Game 7/20 → Winner: B
Game 8/20 → Winner: A
Game 9/20 → Winner: B
Game 10/20 → Winner: A
Game 11/20 → Winner: B
Game 12/20 → Winner: A
Game 13/20 → Winner: B
Game 14/20 → Winner: B
Game 15/20 → Winner: B
Game 16/20 → Winner: B
Game 17/20 → Winner: B
Game 18/20 → Winner: A
Game 19/20 → Winner: B
Game 20/20 → Winner: A

=== Arena Results ===
Model A wins: 7
Model B wins: 13
Win rate A: 0.35
[main] Arena win rate for new model: 0.35
[main] New model rejected. Keeping previous model_old.

[main] ITERATION 2 - self-play 128 games
[main] Generating 128 games concurrently...
[ReplayBuffer] Saved: E:\Projects\GenGameAI\data\replay\batch_0002.npz
[main] Loading replay data to train
[ReplayBuffer] Loaded 2 batches
[main] Training for 3 epochs...

[Trainer] Training on 52641 samples

Epoch 1/3
  Batch   0/822 | Loss: 0.9302 | Policy: 0.8115 | Value: 0.1187
  Batch  20/822 | Loss: 0.6796 | Policy: 0.4755 | Value: 0.2041
  Batch  40/822 | Loss: 0.8391 | P

CancelledError: 

## 5. Download Trained Model
Download the latest model checkpoint.

In [ ]:
from google.colab import files
from pathlib import Path

model_path = Path("data/checkpoints/model_latest.pth")
if model_path.exists():
    files.download(str(model_path))
    print(f"Downloaded: {model_path}")
else:
    print("No model found.")

## 6. Monitoring (Optional)

In [5]:
from pathlib import Path

CHECKPOINT_DIR = Path("data/checkpoints")
REPLAY_DIR = Path("data/replay")

def show_training_status():
    print("=" * 50)
    print("TRAINING STATUS")
    print("=" * 50)
    
    if CHECKPOINT_DIR.exists():
        checkpoints = list(CHECKPOINT_DIR.glob("*.pth"))
        print(f"\nCheckpoints: {len(checkpoints)}")
        for cp in checkpoints:
            size_mb = cp.stat().st_size / (1024 * 1024)
            print(f"  - {cp.name}: {size_mb:.2f} MB")
    
    if REPLAY_DIR.exists():
        replays = list(REPLAY_DIR.glob("*.npz"))
        print(f"\nReplay batches: {len(replays)}")
        if replays:
            total_size = sum(r.stat().st_size for r in replays) / (1024 * 1024)
            print(f"  Total size: {total_size:.2f} MB")
    
    if torch.cuda.is_available():
        print("\nGPU Memory:")
        print(f"  Allocated: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB")
        print(f"  Cached: {torch.cuda.memory_reserved(0) / 1e9:.2f} GB")

show_training_status()

TRAINING STATUS

Checkpoints: 2
  - model_latest.pth: 26.37 MB
  - model_old.pth: 26.37 MB

Replay batches: 13
  Total size: 25.83 MB

GPU Memory:
  Allocated: 0.23 GB
  Cached: 0.38 GB
